
# ****Import Libraries****




In [68]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# ****Data Import****

In [69]:
df = pd.read_csv(r'https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


# ****Check Shape and Drop "id" & "Unnamed: 32"****

In [70]:
df.drop(columns = ['id', 'Unnamed: 32'], inplace = True)
display(df.shape)

(569, 31)

# ****Train Test Split****

In [71]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0])

# ****Standard Scaler****

In [72]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ****Label Encoder****

In [73]:
encode = LabelEncoder()
y_train = encode.fit_transform(y_train)
y_test = encode.transform(y_test)

# ****Convert Numpy Arrays to Pytorch Tensors****

In [74]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

# ****Defining Model****

In [75]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

# ****Important Params****

In [76]:
learning_rate = 0.1
epochs = 100

In [77]:
# Define Loss Function
loss_function = nn.BCELoss()

In [78]:
type(loss_function)

torch.nn.modules.loss.BCELoss

# ****Training Pipeline****

In [79]:
# create model

model = MySimpleNN(X_train_tensor.shape[1])

# Define Optimizer

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# Define Loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train_tensor)

  # Loss Function
  loss = loss_function(y_pred, y_train_tensor.view(-1, 1))

  # Clear Gradients
  optimizer.zero_grad()

  # backward Pass
  loss.backward()

  # Parameter Update
  optimizer.step()

  # Print loss in each epoch
  print(f'Epoch : {epoch + 1}, Loss : {np.round(loss.item(), 4)}')

Epoch : 1, Loss : 0.5134
Epoch : 2, Loss : 0.4374
Epoch : 3, Loss : 0.3884
Epoch : 4, Loss : 0.3537
Epoch : 5, Loss : 0.3274
Epoch : 6, Loss : 0.3067
Epoch : 7, Loss : 0.2897
Epoch : 8, Loss : 0.2756
Epoch : 9, Loss : 0.2635
Epoch : 10, Loss : 0.253
Epoch : 11, Loss : 0.2438
Epoch : 12, Loss : 0.2357
Epoch : 13, Loss : 0.2284
Epoch : 14, Loss : 0.2218
Epoch : 15, Loss : 0.2158
Epoch : 16, Loss : 0.2104
Epoch : 17, Loss : 0.2054
Epoch : 18, Loss : 0.2007
Epoch : 19, Loss : 0.1965
Epoch : 20, Loss : 0.1925
Epoch : 21, Loss : 0.1888
Epoch : 22, Loss : 0.1853
Epoch : 23, Loss : 0.1821
Epoch : 24, Loss : 0.1791
Epoch : 25, Loss : 0.1762
Epoch : 26, Loss : 0.1735
Epoch : 27, Loss : 0.1709
Epoch : 28, Loss : 0.1685
Epoch : 29, Loss : 0.1662
Epoch : 30, Loss : 0.164
Epoch : 31, Loss : 0.162
Epoch : 32, Loss : 0.16
Epoch : 33, Loss : 0.1581
Epoch : 34, Loss : 0.1563
Epoch : 35, Loss : 0.1546
Epoch : 36, Loss : 0.1529
Epoch : 37, Loss : 0.1514
Epoch : 38, Loss : 0.1499
Epoch : 39, Loss : 0.1484


# ****Evaluation****

In [80]:
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy : {accuracy.item()}')

Accuracy : 0.543107271194458


# ****Improvements****

1. `Building the neural network with nn module.`
2. `Using Built-in Activation Function.`
3. `Using Built-in Loss Function.`
4. `Using Built-in Optimizer.`